# Visuzalize AWS Samples Mask-RCNN Detection Results

This notebook visualizes detection results predicted by a trained [AWS Samples Mask-RCNN](https://github.com/aws-samples/mask-rcnn-tensorflow) model. 

## Load trained model

First we define the system path for Python classes.

In [ ]:
import sys
import os
os.environ['TENSORPACK_FP16'] = 'true'
sys.path.append('/mask-rcnn-tensorflow/MaskRCNN')

Next, we initialize the ResNet FPN Mask RCNN model.

In [ ]:
from model.generalized_rcnn import ResNetFPNModel

# create a mask r-cnn model
mask_rcnn_model = ResNetFPNModel(True)

Next we list the log directories to select the directory with a model checkpoint we want to load. The general patterm for the `model_dir` below is `/efs/maskrcnn-opt-yyyy-mm-dd-hh-mm-ss/train_log/maskrcnn/`

In [ ]:
!ls -ltr /efs/maskrcnn-opt-*

Next we specify the `model_dir` below and load the `trained model`. 

In [ ]:
import glob

# find best pre-trained model checkpoint
latest_trained_model = ""
model_dir=  # e.g. "/efs/maskrcnn-opt-2020-02-24-19-48-17/train_log/maskrcnn/"
assert os.path.isdir(model_dir)

model_search_path = os.path.join(model_dir, "model-*.index" )
for model_file in glob.glob(model_search_path):
    if model_file > latest_trained_model:
        latest_trained_model = model_file

trained_model = latest_trained_model[:-6]
print(f'Using trained model: {trained_model}')

Next we initialize the model configuration to match the configuration we used for training.

In [ ]:
from dataset import DetectionDataset
from config import finalize_configs, config as cfg

# setup config
cfg.MODE_FPN = True
cfg.MODE_MASK = True
cfg.DATA.BASEDIR = '/efs/data/'
DetectionDataset()

finalize_configs(is_training=False)

## Create Predictor

Next we create a predictor that uses our trained model to make predictions on test images.

In [ ]:
from tensorpack.predict.base import OfflinePredictor
from tensorpack.tfutils.sessinit import get_model_loader
from tensorpack.predict.config import PredictConfig

# Create an inference predictor           
predictor = OfflinePredictor(PredictConfig(
        model=mask_rcnn_model,
        session_init=get_model_loader(trained_model),
        input_names=['images', 'orig_image_dims'],
        output_names=[
            'generate_{}_proposals_topk_per_image/boxes'.format('fpn' if cfg.MODE_FPN else 'rpn'),
            'generate_{}_proposals_topk_per_image/scores'.format('fpn' if cfg.MODE_FPN else 'rpn'),
            'fastrcnn_all_scores',
            'output/boxes',
            'output/scores',
            'output/labels',
            'output/masks'
        ]))

## Download COCO Test 2017 dataset
Below we download the [COCO 2017 Test dataset](http://cocodataset.org/#download) and extract the downloaded dataset.

In [ ]:
!wget -O /tmp/test2017.zip http://images.cocodataset.org/zips/test2017.zip

In [ ]:
!unzip -q -d /tmp/ /tmp/test2017.zip
!rm  /tmp/test2017.zip

## Define visualization helper functions

Next we define helper functions to visualize the results.

The function `get_mask` resizes the detection mask to the size of the object bounding box and applies the mask to the image.

In [ ]:
from tensorpack.utils.palette import PALETTE_RGB

def get_mask(img, box, mask, threshold=.5):
    box = box.astype(int)
    color = PALETTE_RGB[np.random.choice(len(PALETTE_RGB))][::-1]
    a_mask = np.stack([(cv2.resize(mask, (box[2]-box[0], box[3]-box[1])) > threshold).astype(np.int8)]*3, axis=2)
    sub_image = img[box[1]:box[3],box[0]:box[2],:].astype(np.uint8)
    sub_image = np.where(a_mask==1, sub_image * (1 - 0.5) + color * 0.5, sub_image)
    new_image = img.copy()
    new_image[box[1]:box[3],box[0]:box[2],:] = sub_image
    return new_image


The function `show_detection_results` applies the  masks and bounding boxes to the image and visualizes the detection results.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches

def show_detection_results(img, boxes, scores, labels, masks,  score_threshold=.7, mask_threshold=0.5):
    fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
    
    for bbox, score, label, mask in zip(boxes, scores, labels, masks):
        if score >= score_threshold:
            img = get_mask(img, bbox, mask, mask_threshold)
            
            # Show bounding box
            x1, y1, x2, y2 = bbox
            bbox_y = y1
            bbox_x = x1
            
            bbox_w = (x2 - x1)
            bbox_h = (y2 - y1)

            color = PALETTE_RGB[np.random.choice(len(PALETTE_RGB))][::-1]/255
            box_patch = patches.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, 
                        linewidth=1,
                        alpha=0.7, linestyle="dashed",
                        edgecolor=color, facecolor='none')
            ax.add_patch(box_patch)
            class_name=cfg.DATA.CLASS_NAMES[label]
            ax.text(bbox_x, bbox_y + 8, class_name,
                color='w', size=11, backgroundcolor="none")
            
    ax.imshow(img.astype(int))
    plt.show()


## Load test image

Next, we find a random image to test from COCO 2017 Test dataset. You can come back to this cell when you want to load the next test image.

In [ ]:
import os
import random

test2017_dir=os.path.join("/tmp", "test2017")
img_id=random.choice(os.listdir(test2017_dir))
img_local_path = os.path.join(test2017_dir,img_id)
print(img_local_path)

Next, we load the random test image and convert the image color scheme from BGR to RBG.

In [ ]:
import cv2

img=cv2.imread(img_local_path, cv2.IMREAD_COLOR)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Next. we show the raw image we randomly loaded from the COCO 2017 Test dataset.

In [ ]:
fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(img.astype(int))
plt.show()

## Predict
Next, we use the predictor to predict detection results.

In [ ]:
import numpy as np

_, _, all_scores, final_boxes, final_scores, final_labels, masks = predictor(np.expand_dims(img, axis=0),
                                                            np.expand_dims(np.array(img.shape), axis=0))

## Visualize 
Next, we visualize the detection results on our image.

In [ ]:
show_detection_results(img, final_boxes, final_scores, final_labels, masks)

Go back to <b>Load test image</b> cell if you want to test more images.